In [25]:
import pandas as pd

In [26]:
df = pd.read_csv("pcv_final.csv")

In [27]:
df.head()

,parent,id,Performance Title,composer_text,Concert Title,original_date,date_start,orchestra,Venue Name,Venue Town,seonametown
0,1,101,Der Freischutz; Overture,Weber,1st Concert of the 1st Season,30th Jan 1858,1858-01-30,Hallé Orchestra; 1857-,Free Trade Hall,Manchester,manchester-free-trade-hall
1,1,1798,Symphony No1 in C,Beethoven,1st Concert of the 1st Season,30th Jan 1858,1858-01-30,Hallé Orchestra; 1857-,Free Trade Hall,Manchester,manchester-free-trade-hall
2,1,2987,Symphony in E flat; Andante in A flat,Mozart,1st Concert of the 1st Season,30th Jan 1858,1858-01-30,Hallé Orchestra; 1857-,Free Trade Hall,Manchester,manchester-free-trade-hall
3,1,3950,The Syren; Overture,Auber,1st Concert of the 1st Season,30th Jan 1858,1858-01-30,Hallé Orchestra; 1857-,Free Trade Hall,Manchester,manchester-free-trade-hall
4,1,3952,The Siege of Corinth; Overture,Rossini,1st Concert of the 1st Season,30th Jan 1858,1858-01-30,Hallé Orchestra; 1857-,Free Trade Hall,Manchester,manchester-free-trade-hall


In [28]:
df.nunique()

parent                8149
id                   37217
Performance Title     8145
composer_text         1368
Concert Title          793
original_date         7804
date_start            7784
orchestra               58
Venue Name             254
Venue Town             262
seonametown            378
dtype: int64

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37217 entries, 0 to 37216
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   parent             37217 non-null  int64 
 1   id                 37217 non-null  int64 
 2   Performance Title  37216 non-null  object
 3   composer_text      37202 non-null  object
 4   Concert Title      6098 non-null   object
 5   original_date      37217 non-null  object
 6   date_start         37217 non-null  object
 7   orchestra          37217 non-null  object
 8   Venue Name         37217 non-null  object
 9   Venue Town         37217 non-null  object
 10  seonametown        37217 non-null  object
dtypes: int64(2), object(9)
memory usage: 3.1+ MB


In [30]:
df.drop("Concert Title", axis=1, inplace=True)

In [31]:
df = df[~df.isna().any(axis=1)]

In [32]:
df = df[df["Venue Name"] != 'unlisted']

In [33]:
# Create a nested structure as described
grouped = df.groupby(['Venue Name', 'Venue Town', 'parent', 'date_start'])

# Create a structured dictionary for each group
def make_concert_info(group):
    performances = []
    for _, row in group.iterrows():
        performances.append({
            "title": row['Performance Title'],
            "composer": row['composer_text'],
            "orchestra": row['orchestra']
        })
    return performances

# Apply the function to group data
concerts = grouped.apply(make_concert_info).reset_index(name='performances')
concerts.rename(columns={'parent': 'concert_id', 'date_start': 'date'}, inplace=True)

# Organizing concerts under venues
venues = concerts.groupby(['Venue Name', 'Venue Town']).apply(lambda x: x[['concert_id', 'date', 'performances']].to_dict('records')).reset_index(name='concerts')
print(venues.head())


        Venue Name   Venue Town  \
0       ABC Cinema  Northampton   
1      ABC Theatre    Blackpool   
2     Abbey Cinema      Wexford   
3       Abbey Road       London   
4  Adam Smith Hall     Kirkaldy   

                                            concerts  
0  [{'concert_id': 6162, 'date': '1969-03-02', 'p...  
1  [{'concert_id': 7143, 'date': '1976-06-06', 'p...  
2  [{'concert_id': 4394, 'date': '1958-10-26', 'p...  
3  [{'concert_id': 2555, 'date': '1948-12-02', 'p...  
4  [{'concert_id': 1964, 'date': '1946-06-19', 'p...  


In [34]:
venues

,Venue Name,Venue Town,concerts
0,ABC Cinema,Northampton,"[{'concert_id': 6162, 'date': '1969-03-02', 'p..."
1,ABC Theatre,Blackpool,"[{'concert_id': 7143, 'date': '1976-06-06', 'p..."
2,Abbey Cinema,Wexford,"[{'concert_id': 4394, 'date': '1958-10-26', 'p..."
3,Abbey Road,London,"[{'concert_id': 2555, 'date': '1948-12-02', 'p..."
4,Adam Smith Hall,Kirkaldy,"[{'concert_id': 1964, 'date': '1946-06-19', 'p..."
...,...,...,...
370,Winter Gardens,Bournemouth,"[{'concert_id': 3042, 'date': '1952-06-06', 'p..."
371,Winter Gardens,Eastbourne,"[{'concert_id': 2503, 'date': '1949-05-28', 'p..."
372,Winter Gardens,Malvern,"[{'concert_id': 7500, 'date': '1978-06-04', 'p..."
373,Winter Gardens,Margate,"[{'concert_id': 4939, 'date': '1962-04-29', 'p..."


In [35]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time

# Initialize the geocoder with a specific user_agent
geolocator = Nominatim(user_agent="my_geocode_app")

# Define a rate limiter to manage request rates
geocode_with_delay = RateLimiter(geolocator.geocode, min_delay_seconds=1, error_wait_seconds=10)

def geocode_venue(row):
    try:
        # Combining venue name and town for a more precise geocode
        search_query = f"{row['Venue Name']}, {row['Venue Town']}"
        location = geocode_with_delay(search_query)
        if location:
            print(f"Geocoded: {search_query} -> Latitude: {location.latitude}, Longitude: {location.longitude}")
            return location.latitude, location.longitude
        else:
            print(f"Location not found: {search_query}")
            return None, None
    except Exception as e:
        print(f"Error geocoding {search_query}: {e}")
        return None, None

# Apply the geocoding function to the venues DataFrame
venues[['latitude', 'longitude']] = venues.apply(geocode_venue, axis=1, result_type='expand')

# Display the DataFrame to confirm coordinates are added
print(venues)


Location not found: ABC Cinema, Northampton
Location not found: ABC Theatre, Blackpool
Location not found: Abbey Cinema, Wexford
Geocoded: Abbey Road, London -> Latitude: 51.5325414, Longitude: 0.0036092
Location not found: Adam Smith Hall, Kirkaldy
Geocoded: Albert Hall, Manchester -> Latitude: 53.478299050000004, Longitude: -2.247901788881243
Geocoded: Albert Hall, Nottingham -> Latitude: 52.954312, Longitude: -1.156231121262001
Location not found: Arcadia Theatre, Llandudno
Geocoded: Ashton Hall, Lancaster -> Latitude: 54.0088778, Longitude: -2.8227046930252007
Geocoded: Assembly Hall, Tunbridge Wells -> Latitude: 51.132386, Longitude: 0.2644681
Geocoded: Assembly Hall, Walthamstow -> Latitude: 51.5908778, Longitude: -0.012399405402629649
Location not found: Assembly Hall, Wembley
Geocoded: Assembly Rooms, Derby -> Latitude: 52.923651250000006, Longitude: -1.476435354528702
Location not found: Auditorium der Universitat, Regensburg
Geocoded: BASF, Ludwigshafen -> Latitude: 49.518240

In [40]:
venues.dropna(inplace=True)

In [51]:
# Custom function to format data correctly
def format_data(row):
    return {
        "venue_name": row['Venue Name'],
        "venue_town": row['Venue Town'],
        "latitude": row['latitude'],
        "longitude": row['longitude'],
        "concerts": row['concerts']
    }

# Apply the function and convert to JSON
formatted_data = venues.apply(format_data, axis=1).tolist()
import json
json_output = json.dumps(formatted_data, indent=4)  # indent for pretty printing


In [53]:
# Write JSON data to a file
with open('venue_data.json', 'w') as f:
    f.write(json_output)
